In [1]:
import sys
sys.path.append('/home/peng/git/Machine_learning_for_reliability_analysis/Preprocess')
sys.path.append('/home/peng/git/Machine_learning_for_reliability_analysis/DAlgorithms')
sys.path.append('/home/peng/git/Machine_learning_for_reliability_analysis/QSAR/QM/QM7_2d')

import os,pickle,numpy,nn,copy,scipy,scipy.io
import numpy as np
import pandas as pd

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

Using Theano backend.
Using gpu device 0: GeForce GTX 980 Ti (CNMeM is enabled with initial size: 30.0% of memory, cuDNN 5005)


In [3]:
# --------------------------------------------
# Parameters
# --------------------------------------------
seed  = 3453
#print sys.argv
split = 0 # test split

mb    = 25     # size of the minibatch
hist  = 0.1    # fraction of the history to be remembered

results_path = '/home/peng/git/Machine_learning_for_reliability_analysis/QSAR/QM/REsults/'


dataset = scipy.io.loadmat('/home/peng/Documents/Project_C/QSAR_nlp/qm7.mat')

# --------------------------------------------
# Extract training data::: X is the features, T is the target, P is the index
# --------------------------------------------
P = dataset['P'][range(0,split)+range(split+1,5)].flatten()
X = dataset['X'][P]
T = dataset['T'][0,P]

print np.shape(P)
print np.shape(X)
print np.shape(T)

(5732,)
(5732, 23, 23)
(5732,)


In [4]:
batch_size = 23

data_row , data_col = 23, 23

#channel is the number of color or dimensions of data points
nb_channel = 1


nb_filters = 1

# size of pooling area for max pooling
pool_size = (2, 2)

# convolution kernel size
kernel_size = (3, 3)
input_shape = (1,23,23)

# this is reshape the data into a standard format
X_reshape = X.reshape(X.shape[0], 1, data_row, data_col)

In [5]:
# the stack of layers for CNN

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid', dim_ordering = 'th',
                        input_shape=input_shape))
          
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('linear'))

In [6]:
#compile and predict

model.compile(loss='mae',
              optimizer='sgd',
              metrics=['mae'])

model.fit(X_reshape, T, batch_size=batch_size, nb_epoch=10,
          verbose=1)
score = model.evaluate(X_reshape, T, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
5732/5732 [==============================] - 0s - loss: 1545.8129 - mean_absolute_error: 1545.8129     
Epoch 2/10
5732/5732 [==============================] - 0s - loss: 1224.7819 - mean_absolute_error: 1224.7819     
Epoch 3/10
5732/5732 [==============================] - 0s - loss: 363.1908 - mean_absolute_error: 363.1908     
Epoch 4/10
5732/5732 [==============================] - 0s - loss: 342.5993 - mean_absolute_error: 342.5993     
Epoch 5/10
5732/5732 [==============================] - 0s - loss: 335.2864 - mean_absolute_error: 335.2864     
Epoch 6/10
5732/5732 [==============================] - 0s - loss: 329.8370 - mean_absolute_error: 329.8370     
Epoch 7/10
5732/5732 [==============================] - 0s - loss: 325.9323 - mean_absolute_error: 325.9323     
Epoch 8/10
5732/5732 [==============================] - 0s - loss: 321.4852 - mean_absolute_error: 321.4852     
Epoch 9/10
5732/5732 [==============================] - 0s - loss: 317.6582 - mean_absolute_